In [1]:
import json

import altair as alt
import polars as pl

from the_cheat_down_under.parse import parse

In [2]:
@alt.theme.register("my_theme", enable=True)
def loader():
    with open("theme.json", "r") as f:
        return json.load(f)

In [ ]:
df = parse(folder="../data/run-efforts-table")
df.write_csv("../data/run-efforts-table.csv")

In [ ]:
def get_chart(i) -> alt.LayerChart:
    base = (
        alt.Chart(
            df.with_columns(
                Quarter=pl.col("Lap")
                .qcut(quantiles=4, labels=["Q1", "Q2", "Q3", "Q4"])
                .over("Activity"),
            ).filter(pl.col("Activity") == i)
        )
        .encode(
            x=alt.X("Lap:Q").scale(domain=(0, 120)).title("Lap (km)"),
        )
        .properties(width=1200, height=200)
    )

    return (
        alt.layer(
            base.mark_area(
                interpolate="step-before", clip=True, color="red", opacity=0.4
            ).encode(
                y=alt.Y("HR:Q")
                .scale(zero=False, domain=[70, 170])
                .axis(
                    title="Heart Rate (bpm)",
                    titleColor="red",
                    labelColor="red",
                    tickColor="red",
                    orient="left",
                )
            ),
            base.mark_line(
                interpolate="step-before", clip=True, color="blue"
            ).encode(
                y=alt.Y("Pace:Q")
                .scale(zero=False, domain=[5, 10])
                .axis(
                    title="Pace (min/km)",
                    titleColor="blue",
                    labelColor="blue",
                    tickColor="blue",
                    orient="right",
                )
            ),
        )
        .resolve_scale(y="independent")
        .properties(title=f"Activity {i}")
    )


alt.vconcat(*[get_chart(i) for i in range(1, 36)]).save("laps.png")

In [27]:
(
    alt.Chart(
        df.with_columns(
            Quarter=pl.col("Lap")
            .qcut(quantiles=4, labels=["Q1", "Q2", "Q3", "Q4"])
            .over("Activity")
            .cast(pl.Utf8),
        ).filter(pl.col("Quarter").is_in(["Q2", "Q3"]))
    )
    .mark_point(clip=True, filled=True)
    .encode(
        x=alt.X("HR:Q").scale(zero=False, domain=[70, 170]).title("HR (bpm)"),
        y=alt.Y("Pace:Q")
        .scale(zero=False, domain=[5, 10])
        .title("Pace (min/km)"),
        color=alt.Color("Quarter:N", title="Quarter").scale(
            domain=["Q1", "Q2", "Q3", "Q4"],
            range=["#1f77b4", "#ff7f0e", "#b96113", "#0d4f7e"],
        ),
    )
    .properties(width=200, height=200)
    .facet(facet=alt.Facet("Activity:N"), columns=4)
    .resolve_scale(x="independent")
)

alt.FacetChart(...)